# Create a Rauscher et al. (2002) webnucleo XML file.
The goal of this notebook is to create webnucleo XML files from Rauscher et al. (2002) output.

Begin by installing and importing the necessary python libraries

In [1]:
import sys, io, requests

!{sys.executable} -m pip install --quiet numpy
!{sys.executable} -m pip install --quiet wnutils

import numpy as np
import wnutils.xml as wx

Retrieve the JINA Reaclib nuclear data from [OSF](https://osf.io/kyhbs/).

In [2]:
xml = wx.Xml(io.BytesIO(requests.get('https://osf.io/kyhbs/download').content))
nuc_data = xml.get_nuclide_data()
reac_data = xml.get_reaction_data()

Now define a list of properties for the structure data.

In [3]:
props = ['grid', 'cell outer total mass', 'cell outer radius', 'cell outer velocity', 
         'cell density', 'cell temperature', 'cell pressure', 'cell specific energy', 
         'cell specific entropy', 'cell angular velocity', 'cell A_bar', 'cell Y_e',
         'stability', 'NETWORK', 'neutrons', 'H1', 'He3', 'He4', 'C12', 'N14', 'O16',
         'Ne20', 'Mg24', 'Si28', 'S32', 'Ar36', 'Ca40', 'Ti44', 'Cr48', 'Fe52', 'Fe54',
         'Ni56', 'Fe56', '\'Fe\'']

Define a routine to create zone data from a *Rauscher et al. (2002)* structure file.

In [4]:
def create_zone_structure_data(structure_file):
    x = np.genfromtxt(structure_file, skip_header=2, dtype=str)

    zone_data = {}

    for i in range(x.shape[0]):
        x[i,0] = x[i,0].rstrip(':')
        zone_data[x[i,0]] = {}

    for i in range(x.shape[0]):
        zone_props = {}
        for j in range(x.shape[1]):
            zone_props[props[j]] = x[i,j]
        zone_data[x[i,0]]['properties'] = zone_props
        zone_data[x[i,0]]['mass fractions'] = {}
        
    return zone_data

Now define a routine to retrieve the composition data from the composition file.  Rename neutrons from *nt1* to *n* to be consistent with [webnucleo](https://webnucleo.readthedocs.io) standards.  This routine also removes any zone without mass fraction data.

In [5]:
def update_mass_fractions_in_zone_data(zone_data, composition_file, nuc_data):
    with open(composition_file) as f:
        s = f.readline().split()
        nucs = s[5:]
        nucs = list(map(lambda x: x.replace('nt1', 'n'), nucs))

    y = np.genfromtxt('s15a29.presn_comp', skip_header=1, dtype=str)

    for i in range(y.shape[0]):
        zone = y[i,0]
        if zone in zone_data:
            masses = y[i,3:]
            mass_fractions = {}
            for j in range(len(masses)):
                m = float(masses[j])
                if m > 0 and nucs[j] in nuc_data:
                    tup = (nucs[j], nuc_data[nucs[j]]['z'], nuc_data[nucs[j]]['a'])
                    mass_fractions[tup] = m
            zone_data[zone]['mass fractions'] = mass_fractions
            
    for key, value in dict(zone_data).items():
        if len(value['mass fractions']) == 0:
            del zone_data[key]

Read in the *Rauscher et al. (2002)* data.  The default (*use_default_files = True*) is to download example data from [OSF](https://osf.io/f6tp8/).  To see the download progress, remove the *-s* flag on the curl commands.  

To use other files, download them from [2sn](http://2sn.org/nucleosynthesis) (be sure to unzip them).  You will need a pre-supernova structure file and composition files for the pre-supernova and the post-explosion models.  Download them to your the directory from which you are running this notebook, or upload them to Colab or Binder.  On Colab, click the file tab on the left and upload the data to the sample_data directory.  On Binder, upload them directly to the working directory.  Change *s25a28.presn_structure* to your structure file name, *s25a29.presn_comp* to your pre-supernova composition file name, and *s28a28d.expl_comp* to your post-explosion composition file name.  Change the variable *use_default_files* to *False*, and execute the remaining notebook cells.

In [6]:
use_default_files = True

if use_default_files:
    !curl -o s15a28.presn_structure -J -L -s https://osf.io/tbghn/download
    !curl -o s15a29.presn_comp -J -L -s https://osf.io/fv4xg/download
    !curl -o s15a28d.expl_comp -J -L -s https://osf.io/vxmc3/download
        
    structure_file = 's15a28.presn_structure'
    presn_composition_file = 's15a29.presn_comp'
    expl_composition_file = 's15a28d.expl_comp'

else:
    structure_file = 's25a28.presn_structure'
    presn_composition_file = 's25a29.presn_comp'
    expl_composition_file = 's25a28d.expl_comp'

Create the zone data from the structure file.  

In [7]:
zone_data = create_zone_structure_data(structure_file)

Now retrieve the composition data from the pre-supernova star.  Rename neutrons from *nt1* to *n* to be consistent with [webnucleo](https://webnucleo.readthedocs.io) standards.

In [8]:
update_mass_fractions_in_zone_data(zone_data, presn_composition_file, nuc_data)

Save the presupernova model to XML.  Change *my_presn_output.xml* to your desired XML output name.  The default is to save only a zone data XML file.  To include nuclide and reaction data, change *full_xml* to *True*.

In [9]:
presn_output_file = 'my_presn_output.xml'

full_xml = False

if full_xml:
    new_presn_xml = wx.New_Xml('libnucnet_input')
    new_presn_xml.set_nuclide_data(nuc_data)
    new_presn_xml.set_reaction_data(reac_data)
    new_presn_xml.set_zone_data(zone_data)
else:
    new_presn_xml = wx.New_Xml('zone_data')
    new_presn_xml.set_zone_data(zone_data)

new_presn_xml.write(presn_output_file)

Now create a post-explosion model.  Create a new zone_data with the structure file and update with mass fraction data from the explosion composition data.

In [10]:
zone_data = create_zone_structure_data(structure_file)
update_mass_fractions_in_zone_data(zone_data, expl_composition_file, nuc_data)

Save the post-explosion model to XML.  Change *my_expl_output.xml* to your desired XML output name.  The default is to save only a zone data XML file.  To include nuclide and reaction data, change *full_xml* to *True*.

In [11]:
expl_output_file = 'my_expl_output.xml'

full_xml = False

if full_xml:
    new_expl_xml = wx.New_Xml('libnucnet_input')
    new_expl_xml.set_nuclide_data(nuc_data)
    new_expl_xml.set_reaction_data(reac_data)
    new_expl_xml.set_zone_data(zone_data)
else:
    new_expl_xml = wx.New_Xml('zone_data')
    new_expl_xml.set_zone_data(zone_data)

new_expl_xml.write(expl_output_file)